# Constance de Salm #

Extracting and generating T for CdS. Twtteraccound: https://twitter.com/constancesalm

Importing the needed libraries.

In [1]:
import pandas as pd
import numpy as np
import re
from flash_cards.convert_svg import *

Loading the Dataset via pandas.

In [2]:
df = pd.read_csv("lib/20211116_Constance_de_Salm_Korrespondenz_Inventar_Briefe.csv", sep=';', encoding='utf8')
print(list(df))

['FuD-Key', 'letzte Änderung', 'Dokumenttyp', 'URL', 'Nr.', 'Verfasser', 'GND (Verfasser)', 'VIAF (Verfasser)', ' Empfänger', 'GND (Empfänger)', 'VIAF (Empfänger)', 'Datierung (JJJJ-MM-TT)', 'Alternative Datierung', 'Ausstellungsort', 'Geonames (Ausstellungsort)', 'Empfangsort', 'Geonames (Empfangsort)', 'Vorlage', 'Vollständigkeit des Archivals', 'Erläuterung Unvollständigkeit', 'Umfang in Seiten', 'Anfang des Briefes', 'Inhaltliche Stichpunkte', 'Weitere Personen', 'GND (Weitere Personen)', 'VIAF (Weitere Personen)', 'Orte', 'Geonames (Orte)', 'Schlagwörter', 'Editionen', 'Vermerk', 'Bestand', 'Verweise', 'Digitalisate: Dateiname (Signatur)', 'Zitierempfehlung']


Extracting just the letters which were by Constance de Salm herself.

In [3]:

df_cds = pd.DataFrame(columns=list(df))
rows = []
for i, v in df.iterrows():
    if type(v["Verfasser"]) is not str: continue
    if re.search(r'.*Constance.*', v["Verfasser"]):
        rows.append(v)
df_cds = pd.DataFrame(rows)

## Generating the toots ##
How many toots can be used for the Mastodon-account (based on the length, which is limited by Twitter [500])


In [4]:
length = []
long_toots = []
limit = 500
for i, v in df_cds.iterrows():
    toot = '"{0}..." à {1}, {2} ({3})'.format(v["Anfang des Briefes"],
                                            v[" Empfänger"],
                                            v["Datierung (JJJJ-MM-TT)"],
                                            v["URL"]).replace('\n', '').replace('\t', '')
    length.append(len(toot))
    if len(toot) > limit:
        long_toots.append(toot)
print(f'{len(list(filter(lambda x: x<=280, length)))} toots are shorter then 500 chars.')

2764 toots are shorter then 500 chars.


In [6]:
def generate_next_day(d: int, m: int, y: int):
    months = {1: 31, 2:28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    d = d + 1
    if d > months[m]:
        d = 1
        m +=1
        if m > 12:
            m = 1
            y += 1
    return y, m, d


geo_data = pd.read_csv("lib/CdS_Places.csv", encoding="utf8", index_col='id')
file = open('flash_cards/flashcard_cds.svg', encoding='utf8')
svg_format = file.read()
file.close()
rows = []
day = (2024, 6, 25)
change = 0
for i, v in df_cds.iterrows():
    row = {}
    empf = v[" Empfänger"].split(', ')[0] if type(v[" Empfänger"]) is str else 'N.N.'
    toot = f'"{v["Anfang des Briefes"]}" à {empf}, {v["Datierung (JJJJ-MM-TT)"]} ({v["URL"]})'
    if len(toot) > limit: continue
    row["Toot"] = toot
    row["Date"] = f'{day[2]}.{day[1]}.{day[0]}'
    time_toot = ''
    text = str(v["Anfang des Briefes"]).replace('\n', '').replace('\t', '').replace('  ', ' ').strip() + ' ...'
    # flash_card_text = create_flashcard_text(text)
    # flash_card = svg_format.format(text=flash_card_text,
    #                                date=row['Date'],
    #                                name='à ' + v[" Empfänger"].split(', ')[0] if type(v[" Empfänger"]) is str else 'N.N.')
    # with open('flash_cards/CdS_Formal.svg', 'w', encoding='utf8') as f:
    #     f.write(flash_card)
    h = np.random.randint(7, 11) if change == 0 else np.random.randint(16, 20)
    minute = np.random.randint(0, 59)
    s = np.random.randint(0, 59)
    h = str(h) if len(str(h)) == 2 else "0" + str(h)
    minute = str(minute) if len(str(minute)) == 2 else "0" + str(minute)
    s = str(s) if len(str(s)) == 2 else "0" + str(s)
    row["Time"] = f'{h}:{minute}:{s}'
    row["Picture"] = f'https://owncloud.gwdg.de/index.php/apps/files_sharing/ajax/publicpreview.php?file=%2F%2F{v["FuD-Key"]}.png&c=2026c47172154296b50a936a6d54a4d2&x=1024&y=1024&a=1&t=tb2La6rIG13gzBN'
    rows.append(row)
    # convert_svg('flash_cards/CdS_Formal.svg', f'flash_cards/png/{v["FuD-Key"]}.png')
    if change == 1:
        day = generate_next_day(day[2], day[1], day[0])
        change = 0
    else:
        change += 1
toots_df = pd.DataFrame(rows, columns=["Date", "Time", "Toot", "Picture"])

In [7]:
toots_df

,Date,Time,Toot,Picture
0,25.6.2024,08:54:45,"""Cher et excellent voisin, j'ai remis de jour ...",https://owncloud.gwdg.de/index.php/apps/files_...
1,25.6.2024,18:29:41,"""Je vais retourner à Paris dans 15 jours, Mons...",https://owncloud.gwdg.de/index.php/apps/files_...
2,26.6.2024,09:33:27,"""Nous avons bien regretté, Monsieur, de ne pas...",https://owncloud.gwdg.de/index.php/apps/files_...
3,26.6.2024,16:57:17,"""Je ne vous ai pas encore écrit, Monsieur,"" à ...",https://owncloud.gwdg.de/index.php/apps/files_...
4,27.6.2024,10:45:44,"""Je ne serai jamais embarrassée, Monsieur, qua...",https://owncloud.gwdg.de/index.php/apps/files_...
...,...,...,...,...
2807,29.4.2028,17:28:50,"""Je n’entends point parler de vous, Monsieur, ...",https://owncloud.gwdg.de/index.php/apps/files_...
2808,30.4.2028,10:39:30,"""Vous aurez peut-être été surpris, Monsieur, d...",https://owncloud.gwdg.de/index.php/apps/files_...
2809,30.4.2028,18:45:47,"""Je reçois à l'instant votre lettre de la camp...",https://owncloud.gwdg.de/index.php/apps/files_...
2810,1.5.2028,10:24:30,"""J’ai bien tardé à répondre à votre dernière l...",https://owncloud.gwdg.de/index.php/apps/files_...


In [8]:
toots_df.to_excel("lib/Lettre_de_CdS.xlsx")
toots_df.to_csv("lib/Lettre_de_CdS.tsv", encoding="utf8", sep="\t")

## Creating Flashcards
We start now with the Creation of the Flashcards based on the script found in the directory "flash_cards".
The cards will be saved into the directory: "flash_cards_png".

In [ ]:
day = (2021, 11, 26)
toots_df = pd.DataFrame(columns=["Date", "Time", "Tweet", "Picture", "Longitude", "Lattitude"])
file = open('flash_cards/flashcard_cds.svg', encoding='utf8')
svg_format = file.read()
file.close()
c = 0
for i, v in df_cds.iterrows():
    row = {}
    if type(v["Datierung (JJJJ-MM-TT)"]) is str:
        tmp_date = v["Datierung (JJJJ-MM-TT)"].split('-')
        datierung = '{}.{}.{}'.format(tmp_date[2], tmp_date[1], tmp_date[0]) if len(tmp_date) > 0 else ''
    text = str(v["Anfang des Briefes"]).replace('\n', '').replace('\t', '').replace('  ', ' ').strip() + ' ...'
    # flash_card_text = create_flashcard_text(text)
    # flash_card = svg_format.format(text=flash_card_text,
    #                                date=datierung,
    #                                name='à ' + v[" Empfänger"].split(', ')[0] if type(v[" Empfänger"]) is str else 'N.N.')
    # with open('flash_cards/CdS_Formal.svg', 'w', encoding='utf8') as f:
    #     f.write(flash_card)

    toot = '"{0}..." à {1}, {2} ({3})'.format(str(v["Anfang des Briefes"]),
                                            v[" Empfänger"].split(', ')[0] if type(v[" Empfänger"]) is str else 'N.N.',
                                            v["Datierung (JJJJ-MM-TT)"],
                                            v["URL"]).replace('\n', '').replace('\t', '').replace('  ', ' ').strip()
    row["Tweet"] = toot if len(toot) < 280 else 'Constance de Salm à {}, le {} ({})'.format(v[" Empfänger"].split(', ')[0] if type(v[" Empfänger"]) is str else 'N.N.',
                                                                                              v["Datierung (JJJJ-MM-TT)"],
                                                                                              v["URL"]).replace('\n', '').replace('\t', '').replace('  ', ' ').strip()
    row["Date"] = "{}.{}.{}".format(day[2], day[1], day[0])
    time_toot = ''
    h = np.random.randint(7, 21)
    minute = np.random.randint(0, 59)
    s = np.random.randint(0, 59)
    h = str(h) if len(str(h)) == 2 else "0" + str(h)
    minute = str(minute) if len(str(minute)) == 2 else "0" + str(minute)
    s = str(s) if len(str(s)) == 2 else "0" + str(s)
    row["Time"] = f'{h}:{minute}:{s}'
    if change == 1:
        day = generate_next_day(day[2], day[1], day[0])
        change = 0
    else:
        change += 1
    row["Picture"] = f'https://raw.githubusercontent.com/dhiparis/ConstanceDeSalm/main/CdS_Twitter/flash_cards/png/{v["FuD-Key"]}.png'
    toots_df = toots_df.append(row, ignore_index=True)
    # convert_svg('flash_cards/CdS_Formal.svg', f'flash_cards/png/{v["FuD-Key"]}.png')
    c += 1
    

In [ ]:
toots_df.to_excel("lib/Lettre_de_CdS.xlsx", encoding="utf8")
toots_df.to_csv("lib/Lettre_de_CdS.tsv", encoding="utf8", sep="\t")

## Visualizing the Informations ##

In [ ]:
import folium

In [ ]:
m = folium.Map()
geo_data = pd.read_csv("lib/CdS_Places.csv", encoding="utf8", index_col='id')

for i, v in df_cds.iterrows():
    geo_information = geo_data.loc[geo_data.url == v['Geonames (Ausstellungsort)']]
    try:
        lng = geo_information.iloc[0]['long']
        lat = geo_information.iloc[0]['lat']
        folium.Marker([lat, lng],
                      popup="<i>à {}, le </i>".format(v["Anfang des Briefes"]),
                      tooltip="à {}, le ".format(v[" Empfänger"].split(', ')[0] if type(v[" Empfänger"]) is str else 'N.N.', v["Datierung (JJJJ-MM-TT)"])).add_to(m)
    except IndexError:
        pass